# Data writers in QGL, Auspex, libaps2

It would be nice to have some documentation on how data is stored, processed, and passed around between the three software packages.  I see four situations where data is passed into or out of files.
  * Data written to `.aps2` files by `QGL`
  * Data read by `libaps2` and set to the APS2
  * Data written by `Auspex` into `.auspex` data files
  * Data read by `Auspex`/`Qlab.jl` for analysis

These cover ~99% of the situations I can imagine in our normal data flow.  The rest of this document is an exploration and note-to-future-self about how this process works and how it could be changed in the future.

## QGL to .aps2 files

The first situation will require `QGL` to create sequence files for the APS2.  Here, we'll create these files and read them back into the workspace to see how they are packed. 

In [1]:
import QGL.config
from QGL import *

In [2]:
# a minimal example of a qubit control chain
cl = ChannelLibrary(":memory:")
q2 = cl.new_qubit("q2")

# specify the particulars for a rack of APS2s
ip_addresses = [f"192.168.1.{i}" for i in [23, 24, 25, 28]]
aps2 = cl.new_APS2_rack("Maxwell", ip_addresses, tdm_ip="192.168.1.11")
aps2.px("TDM").trigger_interval = 500e-6
cl.set_master(aps2.px("TDM"))

# initialize all four APS2 to linear regime
for i in range(1,4):
    aps2.tx(i).ch(1).I_channel_amp_factor = 0.5
    aps2.tx(i).ch(1).Q_channel_amp_factor = 0.5
    aps2.tx(i).ch(1).amp_factor = 1

# create a digitizer - X6 in this case
dig_1  = cl.new_X6("MyX6", address=0)
dig_1.record_length = 1024 + 256

AM2 = cl.new_source("AutodyneM2", "HolzworthHS9000", "HS9004A-492-1", 
                     power=16.0, frequency= 6.74621e9, reference="10MHz")
q2src = cl.new_source("q2source", "HolzworthHS9000", "HS9004A-492-2", 
                     power=16.0, frequency=5.0122e9, reference="10MHz")

cl.set_measure(q2, aps2.tx(2), dig_1.channels[1], gate=False, trig_channel=aps2.tx(2).ch("m2"), generator=AM2)
cl.set_control(q2, aps2.tx(4), generator=q2src)


Creating engine...


In [3]:
seqs = [[X(q2),Y(q2),X(q2),MEAS(q2)]]

In [4]:
mf = compile_to_hardware(seqs, 'utils/file_doc')

Compiled 1 sequences.


So now we have `.aps2` files in the utils/file_doc folder, along with a .json metafile.  __mf__ is a file path to the .json metafile that holds metadata about the sequence and where the .aps2 files are stored.  Note, the file extension is set byt the dirver used in QGL.  If you were using an APS1 or a APS3, you would see `.aps1` or `.aps3` files.  Not these files are structurally the same, just name differently for clarity.

In [5]:
mf

'/Users/mware/Github/AWGDir/utils/file_doc-meta.json'

In [6]:
import json

In [7]:
md = json.load(open(mf, "r"))
md

{'axis_descriptor': [{'name': 'segment',
   'partition': 1,
   'points': [1],
   'unit': None}],
 'database_info': {'db_provider': 'sqlite',
  'db_resource_name': ':memory:',
  'library_id': 1,
  'library_name': 'working'},
 'edges': [],
 'instruments': {'Maxwell_U2': '/Users/mware/Github/AWGDir/utils/file_doc-Maxwell_U2.aps2',
  'Maxwell_U4': '/Users/mware/Github/AWGDir/utils/file_doc-Maxwell_U4.aps2'},
 'measurements': ['M-q2'],
 'num_measurements': 1,
 'num_sequences': 1,
 'qubits': ['q2'],
 'receivers': {'RecvChan-MyX6-2': 1}}

In [8]:
APS2_control_file = md["instruments"]["Maxwell_U4"]
APS2_measure_file = md["instruments"]["Maxwell_U2"]

The metafile is just there for house-keeping across multiple `.aps2` files.  It also provides a record of what `QGL` created for each experiment.  The point of this notebook is to dive into the `.aps2` file structure itself.  We can start by looking at the function that creates the file: `QGL/QGL/drivers/APS2Pattern.py` and the `write_sequence_file()`.  The relevant code is only ~20 lines long:
```python
with open(fileName, 'wb') as FID:
    FID.write(b'APS2')                     # target hardware
    FID.write(np.float32(4.0).tobytes())   # Version
    FID.write(np.float32(4.0).tobytes())   # minimum firmware version
    FID.write(np.uint16(2).tobytes())      # number of channels
    # FID.write(np.uint16([1, 2]).tobytes()) # channelDataFor
    FID.write(np.uint64(instructions.size).tobytes()) # instructions length
    FID.write(instructions.tobytes()) # instructions in uint64 form

    #Create the groups and datasets
    for chanct in range(2):
        #Write the waveformLib to file
        if wfInfo[chanct][0].size == 0:
            #If there are no waveforms, ensure that there is some element
            #so that the waveform group gets written to file.
            #TODO: Fix this in libaps2
            data = np.array([0], dtype=np.int16)
        else:
            data = wfInfo[chanct][0]
        FID.write(np.uint64(data.size).tobytes()) # waveform data length for channel
        FID.write(data.tobytes())
```
Here we can see the data is just byte encoded and written to file in a particular order.  First some basic information about the hardware and minimum firmware.

In [9]:
print(b'APS2')

b'APS2'


In [10]:
np.float32(4.0).tobytes()

b'\x00\x00\x80@'

In [11]:
np.uint16(2).tobytes()

b'\x02\x00'

In [12]:
with open(APS2_control_file, "rb") as f:
    print(f.read())

b'APS2\x00\x00\x80@\x00\x00\x80@\x02\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x91\x00\x00\x00\x00\x00/\x00\xa1\x00\x00\x00@\x00a\x00\xa1\x00\x00\x00\x00\x00@\x00!\x00\x00\x00\x05\x00\x00\x00\r\x0b\x00\x00\x00\x00\x01\x00\xa1\x06\x00\x00\x05\x00\x00\x00\r\x00\x00\x00\x05\x00\x00\x00\r#\x00\x00\x00\x00\x01\x00\xa1\x0c\x00\x00\x1d\x00 \x00\r\x00\x00\x00\x00\x00\x00\x00`4\x00\x00\x00\x00\x00\x00\x00t\x01W\x03\xb2\x05\x84\x08\xc2\x0bQ\x0f\x07\x13\xad\x16\x05\x1a\xcf\x1c\xd0\x1e\xdc\x1f\xdc\x1f\xd0\x1e\xcf\x1c\x05\x1a\xad\x16\x07\x13Q\x0f\xc2\x0b\x84\x08\xb2\x05W\x03t\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x004\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Of course, this is just the most basic look at what's in the file.  To pull out more useful information, `QGL` has some other function for exploring the data inside.

In [13]:
QGL.drivers.APS2Pattern.read_sequence_file(APS2_control_file)

{'ch1': [[(1, 0.04541570015871078),
   (1, 0.10438285923574656),
   (1, 0.17800024417043095),
   (1, 0.2661457697472836),
   (1, 0.36747649859602),
   (1, 0.4786961298986693),
   (1, 0.5946770846050543),
   (1, 0.7087046758637529),
   (1, 0.8132096203149799),
   (1, 0.9003784641679893),
   (1, 0.9630081797094372),
   (1, 0.9957270174581858),
   (1, 0.9957270174581858),
   (1, 0.9630081797094372),
   (1, 0.9003784641679893),
   (1, 0.8132096203149799),
   (1, 0.7087046758637529),
   (1, 0.5946770846050543),
   (1, 0.4786961298986693),
   (1, 0.36747649859602),
   (1, 0.2661457697472836),
   (1, 0.17800024417043095),
   (1, 0.10438285923574656),
   (1, 0.04541570015871078),
   (1, 3.1146187425024613e-16),
   (1, -2.5844387463462135e-19),
   (1, -1.2216110335518833e-15),
   (1, 3.911873736643366e-15),
   (1, 2.8801850737119535e-15),
   (1, 4.678017320548365e-16),
   (1, 1.3403101491459762e-14),
   (1, 1.1111039982203213e-14),
   (1, 7.170440821592936e-15),
   (1, 1.7620036993235762e-15),


In [14]:
QGL.drivers.APS2Pattern.read_sequence_file(APS2_measure_file)

{'ch1': [[(24, 0.0),
   (24, 0.0),
   (24, 0.0),
   (1, 0.017946526675619582),
   (1, 0.0894884629471371),
   (1, 0.34550115980954704),
   (1, 0.7392259797338542),
   (1, 0.9383469661823953),
   (1, 0.9877914784519595),
   (1, 0.9976803809058723),
   (1, 0.9995116591380784),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 0.9998779147845196),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
   (1, 1.0),
 

It might be instructive to see what the pulses should look like so we can confirm the data in the files is what was intended.

In [15]:
plot_pulse_files(mf)

Note here, the data returned in the read_sequence_file methods and in plotting is just the waveform data and not the sequence instruction data.  `QGL` has a variety of functions you can use to inspect the sequence data:

In [16]:
QGL.drivers.APS2Pattern.raw_instructions(APS2_control_file)

array([10448491872987906048, 11601324317152903168, 11601379293808033792,
        2377970971995799552,   936748722576949248, 11601273739618025483,
         936748722576949254,   936748722576949248, 11601273739618025507,
         936783907351691276,  6917529027641081856], dtype=uint64)

In [17]:
QGL.drivers.APS2Pattern.read_instructions(APS2_control_file)

[SYNC,
 MODULATION; write=1 | RESET_PHASE; nco_select=0xf,
 MODULATION; write=1 | SET_FREQ; nco_select=0x1, increment=0x40000000,
 WAIT,
 WFM; engine=3, write=1 | PLAY; TA_bit=0, count=5, addr=0,
 MODULATION; write=1 | MODULATE; nco_select=0x1, count=11,
 WFM; engine=3, write=1 | PLAY; TA_bit=0, count=5, addr=6,
 WFM; engine=3, write=1 | PLAY; TA_bit=0, count=5, addr=0,
 MODULATION; write=1 | MODULATE; nco_select=0x1, count=35,
 WFM; engine=3, write=1 | PLAY; TA_bit=1, count=29, addr=12,
 GOTO | target_addr=0]

Users will likely never need to inspect .aps2 files in this detail but the above is illustrative of what information is in the files.  See the documentations for more details on what the instructions actually do when decoded by the hardware.  The above is a representation of our simple X,Y,X sequence created above.

## Data read buy libaps2

The data read by `libaps2` can be specified by the user manually or read from files like `.aps2` above with the `load_sequence_file()` function inside libaps2:
```cpp
// Read the header information
char junk[100];
uint64_t buff_length;
uint16_t num_chans;

std::fstream file(seqFile, std::ios::binary | std::ios::in);
if( !file ) throw APS2_SEQFILE_FAIL;

file.read(junk, 12); // Don't need this info
file.read(reinterpret_cast<char *> (&num_chans), sizeof(uint16_t));

// Start anew
clear_channel_data();

// Read the instructions
file.read(reinterpret_cast<char *> (&buff_length), sizeof(uint64_t));
vector<uint64_t> instructions;
instructions.resize(buff_length);
file.read(reinterpret_cast<char *> (instructions.data()), buff_length*sizeof(uint64_t));
write_sequence(instructions);

// Read the waveforms
vector<int16_t> waveform;
for (int chanct = 0; chanct < num_chans; chanct++) {
  file.read(reinterpret_cast<char *> (&buff_length), sizeof(uint64_t));
  waveform.resize(buff_length);
  file.read(reinterpret_cast<char *> (waveform.data()), buff_length*sizeof(int16_t));
  set_waveform(chanct, waveform);
}
```
To manually set the values a user could use the `write_sequence` function to write sequence instruction information and the `set_waveform` function to write the waveform data.

## Data written by `Auspex`

The other end of the data taking process involves saving experimental data to file.  This data could have a very rich structure and will depend on how the data pipeline was constructed in `Auspex` and what was saved to file using the pipeline `Writer`.  The writer uses a data structure called __AuspexDataContainer__ specified in `auspex/data_format`.  It creates a directory structure with a `.auspex` extension where the data files are stored.  This choice to use directories was intentionally made to allow for multiple file writers to work at once.  The files inside the folder have the following structure:
```
folder/data_set_name/
                        data.dat
                        data_meta.json
```
Where the .dat contains the actual data and the JSON file holds meta data about data in the .dat file.  The data files are `numpy` memmaps as explained in more detail in the next section.

## Data read by `Auspex`

The last situation is where data needs to be read from file after the experiment is over.  This could be necessary for plotting, analysis, etc...  Auspex has two main functions for doing this: `open_data` and `load_data`.

In [18]:
from auspex.analysis.helpers import open_data, load_data

In [19]:
auspex_data = load_data('/Users/mware/test_data.auspex/')

In [20]:
auspex_data

{'q2-rr': {'data': {'data': array([[ 0.14135742-0.06671143j,  0.29278564+0.05685425j,
            0.2131958 +0.03631592j, ..., -0.2923584 +0.09896851j,
           -0.41983032+0.01977539j, -0.31851196-0.08001709j],
          [-0.0007019 -0.08493042j, -0.48803711+0.08099365j,
           -0.01580811-0.19561768j, ...,  0.10314941+0.0663147j ,
           -0.07260132-0.06164551j,  0.03952026+0.07626343j],
          [-0.12911987-0.04852295j, -0.07565308+0.03753662j,
            0.19259644-0.12484741j, ..., -0.00152588-0.09460449j,
           -0.10635376+0.00350952j, -0.14587402-0.28305054j],
          ...,
          [-0.23001099+0.16162109j, -0.105896  -0.22988892j,
           -0.37368774+0.04934692j, ..., -0.25598145+0.15072632j,
           -0.36584473-0.13699341j, -0.14022827+0.18487549j],
          [-0.20065308+0.16748047j, -0.08407593-0.03424072j,
           -0.45407104-0.06930542j, ..., -0.33081055+0.0390625j ,
           -0.21429443+0.05517578j, -0.24783325-0.03045654j],
          [-0.0

In [21]:
dat = open_data('/Users/mware/test_data.auspex/', groupname="q2-main", datasetname="data")
# because no file numer is given, this functions the same as load data

In [22]:
dat

{'q2-rr': {'data': {'data': array([[ 0.14135742-0.06671143j,  0.29278564+0.05685425j,
            0.2131958 +0.03631592j, ..., -0.2923584 +0.09896851j,
           -0.41983032+0.01977539j, -0.31851196-0.08001709j],
          [-0.0007019 -0.08493042j, -0.48803711+0.08099365j,
           -0.01580811-0.19561768j, ...,  0.10314941+0.0663147j ,
           -0.07260132-0.06164551j,  0.03952026+0.07626343j],
          [-0.12911987-0.04852295j, -0.07565308+0.03753662j,
            0.19259644-0.12484741j, ..., -0.00152588-0.09460449j,
           -0.10635376+0.00350952j, -0.14587402-0.28305054j],
          ...,
          [-0.23001099+0.16162109j, -0.105896  -0.22988892j,
           -0.37368774+0.04934692j, ..., -0.25598145+0.15072632j,
           -0.36584473-0.13699341j, -0.14022827+0.18487549j],
          [-0.20065308+0.16748047j, -0.08407593-0.03424072j,
           -0.45407104-0.06930542j, ..., -0.33081055+0.0390625j ,
           -0.21429443+0.05517578j, -0.24783325-0.03045654j],
          [-0.0

In [23]:
dat, desc = open_data(25, '/Users/mware/IpythonNotebooks/explorations/Tomography/', groupname="q2-main", datasetname="data", date="200428")

In [24]:
dat

array([ 0.11757491+8.94790649e-04j,  0.11372945+7.61291504e-04j,
        0.1097108 +3.41381836e-03j,  0.11565305+3.85566711e-03j,
        0.11406972-6.02569580e-05j,  0.11434543+1.45050049e-03j,
        0.12690302+1.96986389e-03j,  0.1236591 -2.21801758e-03j,
        0.12270319+2.78347778e-03j,  0.11674556+2.14483643e-03j,
        0.12332887-3.34823608e-04j,  0.11484883+7.74255371e-03j,
        0.11116234+1.95097351e-03j,  0.11834564+6.18302917e-03j,
        0.12073071+2.30850220e-04j,  0.11195461-1.55059814e-03j,
        0.11595827+2.38290405e-03j,  0.11267494+3.08003235e-03j,
        0.11376691+1.71936035e-03j,  0.11398874+2.71911621e-04j,
        0.11998241-1.44273376e-03j,  0.1172005 +4.31504822e-03j,
        0.12728264-2.39776611e-04j,  0.11995641-1.02386475e-03j,
        0.11623866+3.69152832e-03j,  0.11876387+3.94175720e-03j,
        0.11855029+6.38311768e-03j,  0.11070195+1.48429871e-03j,
        0.1203999 +7.78686523e-04j,  0.11650262+5.32414246e-03j,
        0.11384222+2.7863

In [25]:
desc

<DataStreamDescriptor(num_dims=1, num_points=52)>

The `.dat` files themselves are binary packed memmaps.  They can be accessed amnually with `numpy.memmap`.  

In [26]:
test_file = '/Users/mware/test_data.auspex/q2-main/data.dat'

In [27]:
test_data = np.memmap(test_file, dtype=complex, mode='r')

In [28]:
test_data

memmap([-0.21082539-0.01154156j, -0.21598453-0.0102796j ,
        -0.2122628 -0.00743111j, -0.21295621-0.00911182j,
        -0.21195979-0.01181885j, -0.20728793-0.01669141j,
        -0.20805388-0.0144841j , -0.20547552-0.01401611j,
        -0.2114826 -0.01103287j, -0.20743567-0.01612018j,
        -0.20435995-0.01763492j, -0.20899553-0.01077438j,
        -0.20994795-0.01547096j, -0.21059593-0.01159688j,
        -0.21041017-0.00849313j, -0.20839189-0.01140884j,
        -0.21262625-0.01393439j, -0.21721539-0.01011887j,
        -0.20983708-0.01291982j, -0.21003951-0.01122446j,
        -0.2103922 -0.01056697j, -0.20752765-0.01192252j,
        -0.2089884 -0.00891245j, -0.20406174-0.0132438j ,
        -0.20663705-0.01377797j, -0.20949117-0.01334669j,
        -0.21110732-0.01057185j, -0.20887247-0.01254921j,
        -0.20507274-0.01520171j, -0.21494058-0.01182599j,
        -0.2046523 -0.01158699j, -0.21539362-0.00903612j,
        -0.20796933-0.01711588j, -0.20728712-0.01795599j,
        -0.206

In [29]:
np.size(test_data)

52

Anyone using `Auspex` should never have to work at this level.  This breakdown of the file structures is presented only for reference.